# A Text Mining Approach to Analyze The Cyber Security Related Articles

## PART-3: Text Mining

__Feature Engineering with NLP techniques__


__Importing of Required Libraries__

In [1]:
import nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from autocorrect import Speller
import pickle
import re


import spacy
import en_core_web_sm


import numpy as np
import pandas as pd


import warnings
warnings.filterwarnings("ignore")

__Loading Data__

As mentioned Part-2 that pandas parquet options doesn't support timedelta type. So we need to use __fastparquet__ option, to keep the timedelta type format.

In [2]:
df_sum=pd.read_parquet('df_sum_parquet.gzip',engine='fastparquet')
df_investigator_cyber=pd.read_parquet('df_investigator_cyber_parque.gzip',engine='fastparquet')
df_sum.head(2)

,AwardID,AwardTitle,AwardEffectiveDate,AwardExpirationDate,AwardAmount,ProgramOfficer,Institution_Name,Institution_StateName,Abstract,Year,Award_Duration,AwardAmount_Million,Abstract_Lenght
0,0110599,Collaborative Research Testing Affect Control ...,2001-08-15,2004-07-31,300000.0,Patricia White,University of Arizona,Arizona,The investigators will conduct a series of exp...,2001,1081 days,0.30,2194
1,0112426,Federal Cyber Service Initiative,2001-06-01,2007-06-30,149995.0,Timothy V. Fossum,University of Tulsa,Oklahoma,This program produces a cadre of computer scie...,2001,2220 days,0.15,765


### A-) Word Frequency

__Replacing Of Contraction Words__

In [3]:
#Replace contraction words to alternative forms before tokenization
replacement_patterns = {
    r"won\'t": "will not",
    r"can\'t": "cannot",
    r"i\'m": "i am",
    r"ain\'t": "is not",
    r"(\w+)\'ll": "\g<1> will",
    r"(\w+)n\'t": "\g<1> not",
    r"(\w+)\'ve": "\g<1> have",
    r"(\w+)\'s": "\g<1> is",
    r"(\w+)\'re": "\g<1> are",
    r"(\w+)\'d": "\g<1> would",
    r"&": "and",
    r"<br/>":" "}

df_sum['Abstract']=df_sum['Abstract'].replace(replacement_patterns, regex=True)
df_sum.head(2)

,AwardID,AwardTitle,AwardEffectiveDate,AwardExpirationDate,AwardAmount,ProgramOfficer,Institution_Name,Institution_StateName,Abstract,Year,Award_Duration,AwardAmount_Million,Abstract_Lenght
0,0110599,Collaborative Research Testing Affect Control ...,2001-08-15,2004-07-31,300000.0,Patricia White,University of Arizona,Arizona,The investigators will conduct a series of exp...,2001,1081 days,0.30,2194
1,0112426,Federal Cyber Service Initiative,2001-06-01,2007-06-30,149995.0,Timothy V. Fossum,University of Tulsa,Oklahoma,This program produces a cadre of computer scie...,2001,2220 days,0.15,765


__Merge All Values in Abstract Columns to A Single String__

In [4]:
abstract_list=df_sum['Abstract'].tolist()
abstract_sum=' '.join(abstract_list)
print('Total character number of all Abstract columns is: {}'.format(len(abstract_sum)))

Total character number of all Abstract columns is: 17055166


__Tokenization and LowerCase__

In [5]:
#word tokenization
wrd_list = nltk.word_tokenize(abstract_sum)

#lowercase
wrd_list=[w.lower() for w in wrd_list]
print('Total lowercase words: {}'.format(len(wrd_list)))

Total lowercase words: 2669124


__Remove Numbers__

In [6]:
#remove numbers
wrd_list_alpha=[w for w in wrd_list if w.isalpha()]
print('Total words consist of only alphabets: {}'.format(len(wrd_list_alpha)))

Total words consist of only alphabets: 2318555


__Remove Written Form of Numbers__

In [7]:
#Create numbers in written form for ignoring
num_written= {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', \
             6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten', \
            11: 'eleven', 12: 'twelve', 13: 'thirteen', 14: 'fourteen', \
            15: 'fifteen', 16: 'sixteen', 17: 'seventeen', 18: 'eighteen', \
            19: 'nineteen', 20: 'twenty', 30: 'thirty', 40: 'forty', \
            50: 'fifty', 60: 'sixty', 70: 'seventy', 80: 'eighty', \
            90: 'ninety', 0: 'zero'}

num_list=list(np.arange(0,100))

num2words_list=[]

def num2words(n):
    try:
        return num_written[n]
    except:
        try:
            return num2words(n-n%10) + num2words(n%10)
        except:
            return 'None'

        
for num in num_list:
    num2words_list.append(num2words(num))
    
wrd_list_rm_written_form = [w for w in wrd_list_alpha if w not in num2words_list]
print('Total words consist of only alphabets: {}'.format(len(wrd_list_rm_written_form)))

Total words consist of only alphabets: 2312021


__Remove Stopwords__

In [8]:
#define stopwords list
stopwords_list=nltk.corpus.stopwords.words('english')

#extend the stopwords list
stopwords_list.extend(['cannot','many','much','also','well','better','via'])

wrd_list_rm_stopwords = [w for w in wrd_list_rm_written_form if w not in stopwords_list]
print('Total words after removing stopwords: {}'.format(len(wrd_list_rm_stopwords)))

Total words after removing stopwords: 1414326


__Remove 1-Length Words__

In [9]:
#remove 1-len words
wrd_list_one_len = [w for w in wrd_list_rm_stopwords if len(w)>=2]

print('Total words after removing 1-len words: {}'.format(len(wrd_list_one_len)))

Total words after removing 1-len words: 1413212


__Counting The Words Frequency__

In [10]:
freq_dict_before=nltk.FreqDist(wrd_list_one_len)
print('Total unique words before lemmatization: {}'.format(len(freq_dict_before)))

Total unique words before lemmatization: 32965


In [11]:
print('First 50 frequency words:\n{}'.format(freq_dict_before.most_common(50)))

First 50 frequency words:
[('research', 19624), ('project', 16513), ('data', 12358), ('systems', 9147), ('students', 9052), ('new', 8583), ('science', 7363), ('security', 5927), ('development', 5220), ('system', 5189), ('information', 5066), ('program', 4725), ('support', 4665), ('design', 4638), ('education', 4430), ('engineering', 4424), ('university', 4368), ('network', 4344), ('using', 4343), ('develop', 4295), ('community', 4291), ('learning', 4287), ('use', 4228), ('software', 4049), ('provide', 3953), ('materials', 3869), ('tools', 3679), ('technology', 3587), ('cyberinfrastructure', 3458), ('applications', 3353), ('researchers', 3284), ('broader', 3278), ('computing', 3248), ('work', 3210), ('cybersecurity', 3193), ('infrastructure', 3101), ('control', 3082), ('nsf', 3076), ('computational', 3055), ('computer', 3046), ('scientific', 3045), ('analysis', 3042), ('methods', 3036), ('high', 2969), ('including', 2938), ('used', 2913), ('award', 2896), ('national', 2846), ('impact', 

__Lemmatization__

In [12]:
#Lemmatization
lemmatizer = WordNetLemmatizer()

wrd_list_lemmas = [lemmatizer.lemmatize(w) for w in wrd_list_one_len]

#Recounting the Words Frequency
freq_dict=nltk.FreqDist(wrd_list_lemmas)

print('Total unique words after lemmatization: {}'.format(len(freq_dict)))


Total unique words after lemmatization: 29316


In [13]:
print('First 50 frequency words:\n{}'.format(freq_dict.most_common(50)))

First 50 frequency words:
[('research', 19634), ('project', 18206), ('system', 14336), ('data', 12358), ('student', 10612), ('new', 8583), ('science', 8549), ('network', 6920), ('program', 6462), ('security', 5927), ('technology', 5892), ('community', 5716), ('support', 5579), ('impact', 5509), ('development', 5476), ('university', 5085), ('information', 5066), ('design', 5035), ('model', 4935), ('application', 4789), ('material', 4609), ('tool', 4439), ('education', 4433), ('engineering', 4424), ('using', 4343), ('develop', 4295), ('learning', 4287), ('use', 4228), ('software', 4051), ('provide', 3953), ('approach', 3580), ('process', 3561), ('method', 3535), ('infrastructure', 3523), ('cyberinfrastructure', 3458), ('researcher', 3441), ('computer', 3424), ('resource', 3423), ('study', 3381), ('activity', 3380), ('analysis', 3368), ('work', 3324), ('broader', 3278), ('control', 3252), ('computing', 3248), ('cybersecurity', 3193), ('nsf', 3078), ('computational', 3055), ('scientific', 

__Saving The Result__

In [14]:
df_FreqDist=pd.DataFrame({'Words':list(freq_dict.keys()),'Count':list(freq_dict.values())})
df_FreqDist.sort_values(by=['Count'],ascending=False,inplace=True)
df_FreqDist=df_FreqDist.reset_index(drop=True)
df_FreqDist.to_csv('FreqDist.csv')
df_FreqDist.head(3)

,Words,Count
0,research,19634
1,project,18206
2,system,14336


## B-) Clustering and Topic Modeling

In [15]:
df_sum2=df_sum[['AwardID','Abstract']][:30]
df_sum2.shape

(30, 2)

In [16]:
df_sum2.head(2)

,AwardID,Abstract
0,0110599,The investigators will conduct a series of exp...
1,0112426,This program produces a cadre of computer scie...


__Replacing Of Contraction Words__

In [17]:
replacement_patterns = {
    r"won\'t": "will not",
    r"can\'t": "cannot",
    r"i\'m": "i am",
    r"ain\'t": "is not",
    r"(\w+)\'ll": "\g<1> will",
    r"(\w+)n\'t": "\g<1> not",
    r"(\w+)\'ve": "\g<1> have",
    r"(\w+)\'s": "\g<1> is",
    r"(\w+)\'re": "\g<1> are",
    r"(\w+)\'d": "\g<1> would",
    r"&": "and",
    r"<br/>":" "}

df_sum2['Abstract'].replace(replacement_patterns, regex=True, inplace=True)
df_sum2.head(2)

,AwardID,Abstract
0,0110599,The investigators will conduct a series of exp...
1,0112426,This program produces a cadre of computer scie...


__Tokenization and LowerCase__

In [18]:
#word tokenization and lowercase
df_sum2['Abstract_Tokens'] = df_sum2['Abstract'].str.lower().apply(nltk.word_tokenize)
print('Total words: {}'.format(sum(df_sum2['Abstract_Tokens'].str.len())))
df_sum2.head(2)

Total words: 9647


,AwardID,Abstract,Abstract_Tokens
0,0110599,The investigators will conduct a series of exp...,"[the, investigators, will, conduct, a, series,..."
1,0112426,This program produces a cadre of computer scie...,"[this, program, produces, a, cadre, of, comput..."


__Remove Numbers__

In [19]:
#remove numbers
for n in range(df_sum2.shape[0]):
    df_sum2['Abstract_Tokens'][n]=[w for w in (abstract for abstract in df_sum2['Abstract_Tokens'][n]) if w.isalpha()]
print('Total words consist of only alphabets: {}'.format(sum(df_sum2['Abstract_Tokens'].str.len())))
df_sum2.head(2)

Total words consist of only alphabets: 8325


,AwardID,Abstract,Abstract_Tokens
0,0110599,The investigators will conduct a series of exp...,"[the, investigators, will, conduct, a, series,..."
1,0112426,This program produces a cadre of computer scie...,"[this, program, produces, a, cadre, of, comput..."


__Remove Written Form of Numbers__

In [20]:
#Create numbers in written form for ignoring
num_written= {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', \
             6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten', \
            11: 'eleven', 12: 'twelve', 13: 'thirteen', 14: 'fourteen', \
            15: 'fifteen', 16: 'sixteen', 17: 'seventeen', 18: 'eighteen', \
            19: 'nineteen', 20: 'twenty', 30: 'thirty', 40: 'forty', \
            50: 'fifty', 60: 'sixty', 70: 'seventy', 80: 'eighty', \
            90: 'ninety', 0: 'zero'}

num_list=list(np.arange(0,100))

num2words_list=[]

def num2words(n):
    try:
        return num_written[n]
    except:
        try:
            return num2words(n-n%10) + num2words(n%10)
        except:
            return 'None'

        
for num in num_list:
    num2words_list.append(num2words(num))
    
df_sum2['Abstract_Tokens'] = df_sum2['Abstract_Tokens'].replace([w for w in num2words_list],'')

print('Total words consist of only alphabets: {}'.format(sum(df_sum2['Abstract_Tokens'].str.len())))

Total words consist of only alphabets: 8325


__Remove Stopwords__

In [21]:
#define stopwords list
stopwords_list=nltk.corpus.stopwords.words('english')

#extend the stopwords list
stopwords_list.extend(['cannot','many','much','also','well','better','via'])

for n in range(df_sum2.shape[0]):
    df_sum2['Abstract_Tokens'][n]=[w for w in (abstract for abstract in df_sum2['Abstract_Tokens'][n]) if w not in stopwords_list]
print('Total words after removing stopwords: {}'.format(sum(df_sum2['Abstract_Tokens'].str.len())))
df_sum2.head(2)

Total words after removing stopwords: 5044


,AwardID,Abstract,Abstract_Tokens
0,0110599,The investigators will conduct a series of exp...,"[investigators, conduct, series, experiments, ..."
1,0112426,This program produces a cadre of computer scie...,"[program, produces, cadre, computer, scientist..."


__Remove 1-Length Words__

Punctuation
Punctuation are the unnecessary symbols that are in our corpus documents,

In [22]:
#remove 1-len words
for n in range(df_sum2.shape[0]):
    df_sum2['Abstract_Tokens'][n]=[w for w in (abstract for abstract in df_sum2['Abstract_Tokens'][n]) if len(w)>=2]
print('Total words after removing 1-len words: {}'.format(sum(df_sum2['Abstract_Tokens'].str.len())))
df_sum2.head(2)

Total words after removing 1-len words: 5037


,AwardID,Abstract,Abstract_Tokens
0,0110599,The investigators will conduct a series of exp...,"[investigators, conduct, series, experiments, ..."
1,0112426,This program produces a cadre of computer scie...,"[program, produces, cadre, computer, scientist..."


__Lemmatization__

In [23]:
#Lemmatization
lemmatizer = WordNetLemmatizer()

for n in range(df_sum2.shape[0]):
    df_sum2['Abstract_Tokens'][n]=[w for w in (abstract for abstract in df_sum2['Abstract_Tokens'][n])]

print('Total words after lemmatization: {}'.format(sum(df_sum2['Abstract_Tokens'].str.len())))
df_sum2.head(2)

Total words after lemmatization: 5037


,AwardID,Abstract,Abstract_Tokens
0,0110599,The investigators will conduct a series of exp...,"[investigators, conduct, series, experiments, ..."
1,0112426,This program produces a cadre of computer scie...,"[program, produces, cadre, computer, scientist..."


In [24]:
#Recounting the Words Frequency
freq_dict=nltk.FreqDist(wrd_list_lemmas)

print('Total unique words after lemmatization: {}'.format(len(freq_dict)))


Total unique words after lemmatization: 29316
